# DATASET >> https://universe.roboflow.com/force-iaope/cars-egrgd/dataset/1#

# PRÉ-AMBULO - IMPORT LIBS

In [ ]:
from ultralytics import YOLO
from threading import Thread
from PIL import Image
import numpy as np
import queue as q
import shutil
import cv2
import os

from modules.helper.helper import create_video_from_images, get_framerate

import torch
import torchvision
print("torch:", torch.__version__)
print("torchvision:", torchvision.__version__)
print("CUDA disponível:", torch.cuda.is_available())

# https://docs.ultralytics.com/datasets/

torch: 2.6.0+cu118
torchvision: 0.21.0+cu118
CUDA disponível: False


# TRAIN

In [ ]:
# Load a pretrained YOLO model (recommended for training)
model = YOLO("../../runs/train/yolo11n_576_1000_v2/weights/epoch550.pt")
# model = YOLO("bast_model\\weights\\epoch550.pt")

# Continue training the model from where it stopped
results = model.train(
    data="data_train.yaml",        # Arquivo de configuração do seu dataset
    epochs=1000,                   # Número de épocas (pode ser ajustado conforme necessário)
    device="cuda" if torch.cuda.is_available() else "cpu",  # Verifica se há GPU disponível, caso contrário usa CPU
    workers=4,                     # Número de workers para carregamento dos dados (aumentado para agilizar)
    imgsz=576,                     # Tamanho das imagens (960x960)
    batch=4,                       # Tamanho do batch (aumentado para melhor estabilidade, se sua GPU suportar)
    cache="disk",                  # Utiliza cache no disco para garantir resultados determinísticos
    save_period=5,                 # Salva o modelo a cada 5 épocas
    name="yolo11n_576_1000_v2",    # Nome da pasta de treinamento
    exist_ok=True,                 # Sobrescreve o diretório existente, se houver
    project="../../runs/train",    # Diretório onde os resultados serão salvos
    resume=True,                   # Continua o treinamento de onde parou
    patience=100,                  # Aumenta a paciência do EarlyStopping para 50 épocas (ajuste conforme necessário)
    optimizer="AdamW",             # Otimizador SGD Adam AdamW pode melhorar a convergência
    pretrained=True,               # Utiliza pesos pré-treinados
    optimize=True,                 # Otimiza o modelo para melhor desempenho
    classes=[1, 2, 3, 4],          # Escolhe as classes a serem treinadas (0: pessoas, 1: carros)
    # hyp="data/hyp.scratch-low.yaml" # Arquivo de hiperparâmetros; ajuste-o conforme o seu dataset
)

# Validate the model on the validation set
valid = model.val()

# Export the model to ONNX format
success = model.export(format="onnx", dynamic=True)


New https://pypi.org/project/ultralytics/8.3.107 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.105  Python-3.12.7 torch-2.6.0+cu118 CPU (13th Gen Intel Core(TM) i7-1355U)
engine\trainer: task=detect, mode=train, model=..\..\runs\train\yolo11n_576_1000_v2\weights\epoch550.pt, data=data_train.yaml, epochs=1000, time=None, patience=100, batch=4, imgsz=576, save=True, save_period=5, cache=disk, device=cpu, workers=0, project=../../runs/train, name=yolo11n_576_1000_v2, exist_ok=True, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=..\..\runs\train\yolo11n_576_1000_v2\weights\epoch550.pt, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augm

train: Scanning C:\dev\datasets\data_train\dataset\train\labels.cache... 365 images, 0 backgrounds, 0 corrupt: 100%|██████████| 365/365 [00:00<?, ?it/s]

train: WARNING  C:\dev\datasets\data_train\dataset\train\images\13813ee3-results_0078.jpg: 1 duplicate labels removed



train: Caching images (0.9GB Disk): 100%|██████████| 365/365 [00:00<00:00, 16186.86it/s]
val: Scanning C:\dev\datasets\data_train\dataset\val\labels.cache... 92 images, 0 backgrounds, 0 corrupt: 100%|██████████| 92/92 [00:00<?, ?it/s]
val: Caching images (0.2GB Disk): 100%|██████████| 92/92 [00:00<00:00, 26241.14it/s]

Plotting labels to ..\..\runs\train\yolo11n_576_1000_v2\labels.jpg... 


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


   638/1000         0G     0.7599     0.4578     0.8425          4        576: 100%|██████████| 92/92 [02:12<00:00,  1.44s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 12/12 [00:06<00:00,  1.73it/s]

                   all         92        848      0.805      0.694      0.743      0.486



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


   639/1000         0G     0.7729     0.4589     0.8478         54        576:  52%|█████▏    | 48/92 [01:06<01:01,  1.39s/it]

In [ ]:
# # Create a new YOLO model from scratch
# model = YOLO("yolo11n.yaml")

# Load a pretrained YOLO model (recommended for training)
# model = YOLO("yolo11n.pt")
model = YOLO("bast_model\\weights\\best.pt")

# Train the model using the 'coco8.yaml' dataset for 3 epochs # c:\dev\datasets
results = model.train(
    data="data_train.yaml",        # Arquivo de configuração do seu dataset
    epochs=1000,                   # Número de épocas (pode ser ajustado conforme necessário)
    device="cuda" if torch.cuda.is_available() else "cpu",  # Verifica se há GPU disponível, caso contrário usa CPU
    workers=4,                     # Número de workers para carregamento dos dados (aumentado para agilizar)
    imgsz=576,                     # Tamanho das imagens (960x960)
    batch=4,                       # Tamanho do batch (aumentado para melhor estabilidade, se sua GPU suportar)
    cache="disk",                  # Utiliza cache no disco para garantir resultados determinísticos
    save_period=5,                 # Salva o modelo a cada 5 épocas
    name="yolo11n_576_1000_v2",     # Nome da pasta de treinamento
    exist_ok=True,                 # Sobrescreve o diretório existente, se houver
    project="../../runs/train",    # Diretório onde os resultados serão salvos
    resume=False,                  # Não retoma de um checkpoint anterior
    patience=100,                  # Aumenta a paciência do EarlyStopping para 50 épocas (ajuste conforme necessário)
    optimizer="AdamW",             # Otimizador SGD Adam AdamW pode melhorar a convergência
    pretrained=True,               # Utiliza pesos pré-treinados
    optimize=True,                 # Otimiza o modelo para melhor desempenho
    classes=[1, 2, 3, 4],          # Escolhe as classes a serem treinadas (0: pessoas, 1: carros)
    # hyp="data/hyp.scratch-low.yaml" # Arquivo de hiperparâmetros; ajuste-o conforme o seu dataset
)

# Validate the model on the validation set
valid = model.val()

# Export the model to ONNX format
success = model.export(format="onnx", dynamic=True)


# TEST IN VIDEO

In [ ]:
# seleção das linhas no início do vídeo
def select_lines(frame):
    lines = {}
    current_line = []
    line_id = 0

    def draw_line(event, x, y, flags, param):
        nonlocal current_line, line_id
        if event == cv2.EVENT_LBUTTONDOWN:
            current_line.append((x, y))
            if len(current_line) == 2:
                lines[f"line_{line_id}"] = {
                    "points": tuple(current_line),
                    "count": 0
                }
                line_id += 1
                current_line = []

    clone = frame.copy()
    cv2.namedWindow("Select Lines")
    cv2.setMouseCallback("Select Lines", draw_line)

    print("Clique em dois pontos para cada linha. Pressione 'q' para finalizar.")

    while True:
        temp_frame = clone.copy()
        for line in lines.values():
            cv2.line(temp_frame, line["points"][0], line["points"][1], (0, 255, 0), 2)
        cv2.imshow("Select Lines", temp_frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cv2.destroyWindow("Select Lines")
    return lines

# Função para atualizar contagem com verificação de múltiplas contagens por ID
def update_count(results, lines_dict, memory, crossed):
    for result in results:
        boxes = result.boxes
        if boxes.id is None:
            continue

        for i, box in enumerate(boxes.xyxy):
            obj_id = int(boxes.id[i])
            x1, y1, x2, y2 = map(int, box)
            center = ((x1 + x2) // 2, (y1 + y2) // 2)

            if obj_id not in memory:
                memory[obj_id] = center
                continue

            prev_center = memory[obj_id]
            memory[obj_id] = center

            if obj_id not in crossed:
                crossed[obj_id] = set()

            for line_key, line_info in lines_dict.items():
                (xA, yA), (xB, yB) = line_info["points"]

                def ccw(A, B, C):
                    return (C[1]-A[1]) * (B[0]-A[0]) > (B[1]-A[1]) * (C[0]-A[0])

                def intersect(A, B, C, D):
                    return ccw(A, C, D) != ccw(B, C, D) and ccw(A, B, C) != ccw(A, B, D)

                if intersect(prev_center, center, (xA, yA), (xB, yB)):
                    if line_key not in crossed[obj_id]:  # evita múltiplas contagens
                        lines_dict[line_key]["count"] += 1
                        crossed[obj_id].add(line_key)


In [ ]:

# Load the YOLO11 model
model = YOLO("../../runs/train/yolo11n_576_1000_v2/weights/best.pt")

# Open the video file
video_path      = "input\\Teste.mp4"
cap = cv2.VideoCapture(video_path)

# Path to save the output frames
save            = "output\\detect"
shutil.rmtree(save)
os.makedirs(save, exist_ok=True)

# Initialize the count frame index and queue
idx = 0
queue = q.Queue(maxsize=100)  # Create a queue with a maximum size of 500

# Function to save the frame as an image
StopFrame = False
def SaveFrame():
    while not StopFrame:
        try:
            frame, idx = queue.get(timeout=0.5)  # Get the frame and index from the queue
            cv2.imwrite(os.path.join(save, f"frame_{idx}.jpg"), frame)
        except Exception as e:
            continue

# Create and start threads for saving frames
SaveFrameThreads = []
for i in range(3):
    thread = Thread(target=SaveFrame, daemon=True)  # Create a thread to save frames
    thread.start()  # Start the thread
    SaveFrameThreads.append(thread)  # Add the thread to the list

# Loop through the video frames
STARTED = True
tracked_ids_memory = {}  # fora do loop.
crossed = {}  # <- novo dicionário para evitar múltiplas contagens
while cap.isOpened():
    
    # Read a frame from the video
    success, frame = cap.read()
    
    if success:
        
        if STARTED:
            STARTED = False
            # Solicita onde contar (linha de contagem)
            lines_dict = select_lines(frame)
            # Resetar posição do vídeo
            cap.set(cv2.CAP_PROP_POS_FRAMES, 0)

        try:
            # Run YOLO11 tracking on the frame, persisting tracks between frames
            results = model.track(
                frame, 
                persist=True,
                # classes=[1, 2, 3, 4],   # Classes to track
                # conf=0.5,               # Confidence threshold
                # iou=0.45,               # IoU threshold
                device= (
                    "cuda" 
                    if torch.cuda.is_available() 
                    else 
                    "cpu"
                ),                      # Verifica se há GPU disponível, caso contrário usa CPU
                show=False,             # Show the results
                save=False,             # Save the results
                save_txt=False,         # Save results to text files
                save_conf=False,        # Save confidence scores
                save_crop=False,        # Save cropped images of detected objects
                # save_dir=save,        # Directory to save results
                # project=save,         # Project name
                name="detect",          # Name of the folder to save results
                exist_ok=True,          # Overwrite existing results
                line_width=1,           # Line thickness for bounding boxes
                show_labels=True,       # Hide labels on bounding boxes
                show_conf=True,         # Hide confidence scores on bounding boxes
                half=True,              # Use half precision (FP16) for inference
                dnn=False,              # Use OpenCV DNN for inference
                vid_stride=1,           # Frame stride for video processing
                agnostic_nms=False,     # Use class-agnostic NMS
                augment=False,          # Use augmentation during inference
                visualize=False,        # Visualize the results
                verbose=False,          # Verbose output
            )
            
            # dentro do loop, depois de results:
            update_count(results, lines_dict, tracked_ids_memory, crossed)

            # Visualize the results on the frame
            annotated_frame = results[0].plot()
            if annotated_frame is None:
                print(f"Resultado {idx} não contém dados válidos para plotar.")
                continue
            
            # Opcional: desenhar linhas e contagens no frame
            for key, data in lines_dict.items():
                pt1, pt2 = data["points"]
                cv2.line(annotated_frame, pt1, pt2, (0, 255, 255), 2)
                mid = ((pt1[0] + pt2[0]) // 2, (pt1[1] + pt2[1]) // 2)
                cv2.putText(annotated_frame, f"{key}: {data['count']}", mid,
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

            # im_rgb = Image.fromarray(annotated_frame[..., ::-1])  # RGB-order PIL image
            queue.put((annotated_frame, idx))  # Add the frame to the queue for saving
            idx += 1  # Increment the frame index
        except ValueError as e:
            print(f"Erro ao processar o resultado {idx}: {e}")
        except Exception as e:
            print(f"Erro inesperado ao processar o resultado {idx}: {e}")

    else:
        # Break the loop if the end of the video is reached
        break

# Wait for all threads to finish processing
StopFrame = True
for idx, thread in enumerate(SaveFrameThreads):
    print(f"Wait for the thread to finish {idx}")
    thread.join()  # Wait for all threads to finish

# Release the video capture object and close the display window
cap.release()

# Exemplo de uso
image_folder = save
output_video_path = "output_video.mp4"
frame_rate = get_framerate(video_path)  # Substitua pelo frame rate desejado
create_video_from_images(image_folder, output_video_path, frame_rate)

# Abrir vídeo automaticamente após criação
os.system(f"start {output_video_path}")